In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
fig_dir = (Path().cwd().parents[0] / 'figure').absolute()

df_meta_path = data_dir / '9PPI Cell Culture' / 'Whole' / 'metadata' /  'imgs_reg_norm.csv'
df_imgs = pd.read_csv(df_meta_path)

# Read per cell data

In [5]:
# Read PPI dfs
PPI_save_path =  data_dir / '9PPI Cell Culture' / 'Whole' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

# Group by location (Cell vs Nuclei)
g = df.groupby(['Condition', 'FOV', 'PPI', 'Cyto']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell = df_cell[df_cell.Cyto != 0]
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']

g = df.groupby(['Condition', 'FOV', 'PPI', 'Nuclei']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei= df_nuclei[df_nuclei.Nuclei != 0]
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']

# Concat 
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei'] # Cyto count

# Ranme columns
df_all.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Cell', 'Nuclei',
       'Cyto']

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,12403.000000,12403.000000,12403.000000,12403.000000
mean,205.029993,47.493832,16.249456,31.244376
std,123.872364,81.222652,24.820231,60.451342
min,5.000000,1.000000,0.000000,-19.000000
25%,99.000000,4.000000,1.000000,2.000000
50%,197.000000,20.000000,6.000000,11.000000
75%,298.000000,51.000000,20.000000,31.000000
max,519.000000,1661.000000,265.000000,1439.000000


,Cell,Nuclei,Cyto
count,1491.000000,1491.000000,1491.000000
mean,395.081154,135.172368,259.908786
std,221.626206,67.238884,179.832845
min,48.000000,0.000000,-26.000000
25%,244.000000,95.000000,137.000000
50%,349.000000,127.000000,222.000000
75%,491.000000,165.000000,334.000000
max,2728.000000,755.000000,2233.000000


In [6]:
df_all = df_all[~df_all.PPI.isin(['Cyclin D1 & CDK2 - re', 'NF-Kb & p-P90rsk - re'])]

In [7]:
# Filter out by maximum number of counts per cell
min_count = 100
max_count = 400

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cell'] < max_count).all())
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cyto'] >= 0).all())

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,9388.000000,9388.000000,9388.000000,9388.000000
mean,205.093098,47.721027,17.328931,30.392096
std,121.681055,69.777520,23.707924,49.704202
min,5.000000,1.000000,0.000000,0.000000
25%,103.000000,4.000000,1.000000,2.000000
50%,198.000000,23.000000,8.000000,13.000000
75%,296.000000,55.000000,23.000000,33.000000
max,519.000000,399.000000,183.000000,369.000000


,Cell,Nuclei,Cyto
count,1328.000000,1328.000000,1328.000000
mean,337.353163,122.503012,214.850151
std,142.524238,50.670961,117.761901
min,102.000000,0.000000,25.000000
25%,228.000000,90.000000,126.000000
50%,316.000000,119.000000,196.000000
75%,427.000000,151.250000,287.000000
max,1202.000000,412.000000,1202.000000


# ML model

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
import wandb

In [9]:
condition = 'ML'
project_name = f'PLA_9PPI_070323_{condition}'

In [10]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count_cell = df_count.iloc[:, df_count.columns.get_level_values(0)=='Cell']

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [11]:
df_count

Cell                                               \
PPI                Bim & Tom20 Cyclin D1 & CDK2 Cyclin E & CDK4 Mcl-1 & BAK   
Condition  FOV Id                                                             
HCC827Ctrl FW1 15         20.0            185.0             3.0         1.0   
               16          7.0            103.0             0.0         2.0   
               17         23.0            121.0             0.0         0.0   
               18         57.0            225.0             2.0         0.0   
               19         26.0            139.0             0.0         0.0   
...                        ...              ...             ...         ...   
HCC827Osim FW2 340        39.0             85.0             0.0         0.0   
               341        17.0            151.0             3.0         1.0   
               343        39.0            218.0             2.0         0.0   
               344        22.0            206.0             1.0         0.0   
               345        28.0            195.0             0.0         0.0   

                                                                          \
PPI                NF-Kb & p-P90rsk P-AKT & mTOR Sox2 & Oct4 TEAD1 & YAP   
Condition  FOV Id                                                          
HCC827Ctrl FW1 15              29.0          2.0        39.0         3.0   
               16              37.0          0.0        19.0         2.0   
               17              35.0          0.0        36.0         1.0   
               18              57.0          0.0        65.0         0.0   
               19              34.0          0.0        39.0         0.0   
...                             ...          ...         ...         ...   
HCC827Osim FW2 340             36.0          0.0        37.0         0.0   
               341             48.0          6.0        60.0         2.0   
               343             80.0          0.0        66.0         0.0   
               344             62.0          0.0        40.0         1.0   
               345             45.0          0.0        58.0         0.0   

                                      Nuclei  ...                      Cyto  \
PPI                p-ERK & c-MYC Bim & Tom20  ... p-ERK & c-MYC Bim & Tom20   
Condition  FOV Id                             ...                             
HCC827Ctrl FW1 15           31.0         5.0  ...           5.0        15.0   
               16           22.0         1.0  ...           4.0         6.0   
               17           15.0        16.0  ...           8.0         7.0   
               18            8.0        40.0  ...           4.0        17.0   
               19           21.0        18.0  ...           2.0         8.0   
...                          ...         ...  ...           ...         ...   
HCC827Osim FW2 340           3.0        20.0  ...           1.0        19.0   
               341           6.0         2.0  ...           2.0        15.0   
               343           6.0        11.0  ...           3.0        28.0   
               344           8.0        10.0  ...           2.0        12.0   
               345           6.0        17.0  ...           3.0        11.0   

                                                                 \
PPI                Cyclin D1 & CDK2 Cyclin E & CDK4 Mcl-1 & BAK   
Condition  FOV Id                                                 
HCC827Ctrl FW1 15             120.0             3.0         1.0   
               16              67.0             0.0         0.0   
               17              83.0             0.0         0.0   
               18             166.0             1.0         0.0   
               19              81.0             0.0         0.0   
...                             ...             ...         ...   
HCC827Osim FW2 340             69.0             0.0         0.0   
               341             92.0             0.0         0.0   
          

In [12]:
df_count.reset_index().groupby('Condition').size()

Condition
HCC827Ctrl    723
HCC827Osim    605
dtype: int64

In [25]:
df_count_cyto_nuclei

Nuclei                                               \
PPI                Bim & Tom20 Cyclin D1 & CDK2 Cyclin E & CDK4 Mcl-1 & BAK   
Condition  FOV Id                                                             
HCC827Ctrl FW1 15          5.0             65.0             0.0         0.0   
               16          1.0             36.0             0.0         2.0   
               17         16.0             38.0             0.0         0.0   
               18         40.0             59.0             1.0         0.0   
               19         18.0             58.0             0.0         0.0   
...                        ...              ...             ...         ...   
HCC827Osim FW2 340        20.0             16.0             0.0         0.0   
               341         2.0             59.0             3.0         1.0   
               343        11.0             81.0             0.0         0.0   
               344        10.0             40.0             1.0         0.0   
               345        17.0             60.0             0.0         0.0   

                                                                          \
PPI                NF-Kb & p-P90rsk P-AKT & mTOR Sox2 & Oct4 TEAD1 & YAP   
Condition  FOV Id                                                          
HCC827Ctrl FW1 15               8.0          0.0        11.0         1.0   
               16              12.0          0.0        10.0         2.0   
               17              18.0          0.0        14.0         0.0   
               18              20.0          0.0        39.0         0.0   
               19              10.0          0.0        14.0         0.0   
...                             ...          ...         ...         ...   
HCC827Osim FW2 340              7.0          0.0         8.0         0.0   
               341             21.0          6.0        39.0         2.0   
               343             40.0          0.0        47.0         0.0   
               344             21.0          0.0        20.0         1.0   
               345             27.0          0.0        27.0         0.0   

                                        Cyto                                   \
PPI                p-ERK & c-MYC Bim & Tom20 Cyclin D1 & CDK2 Cyclin E & CDK4   
Condition  FOV Id                                                               
HCC827Ctrl FW1 15            5.0        15.0            120.0             3.0   
               16            4.0         6.0             67.0             0.0   
               17            8.0         7.0             83.0             0.0   
               18            4.0        17.0            166.0             1.0   
               19            2.0         8.0             81.0             0.0   
...                          ...         ...              ...             ...   
HCC827Osim FW2 340           1.0        19.0             69.0             0.0   
               341           2.0        15.0             92.0             0.0   
               343           3.0        28.0            137.0             2.0   
               344           2.0        12.0            166.0             0.0   
               345           3.0        11.0            135.0             0.0   

                                                                          \
PPI                Mcl-1 & BAK NF-Kb & p-P90rsk P-AKT & mTOR Sox2 & Oct4   
Condition  FOV Id                                                          
HCC827Ctrl FW1 15          1.0             21.0          2.0        28.0   
               16          0.0             25.0          0.0         9.0   
               17          0.0             17.0          0.0        22.0   
               18          0.0             37.0          0.0        26.0   
               19          0.0             24.0          0.0        25.0   
...                        ...              ...          ...         ...   
HCC827Osim FW2 340         0.0   

In [26]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True), 
}

In [27]:
# Run model on cell count
X = df_count_cell
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cell', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.71429
auc,0.71349
b_accuracy,0.71349
f1,0.68852


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.69549
auc,0.69549
b_accuracy,0.69549
f1,0.66939


...

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.72556
auc,0.70373
b_accuracy,0.70373
f1,0.63317


wandb: 
wandb: Plotting SVM_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.71698
auc,0.70833
b_accuracy,0.70833
f1,0.66368


wandb: 
wandb: Plotting SVM_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.67925
auc,0.67746
b_accuracy,0.67746
f1,0.65306


In [28]:
# Run model on cell count
X = df_count_cyto_nuclei
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cyto_nuclei', name=model_name+f'_cyto_nuclei_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cyto_nuclei', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.74812
auc,0.7465
b_accuracy,0.7465
f1,0.72199


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.7406
auc,0.7406
b_accuracy,0.7406
f1,0.71605


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


...

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.76316
auc,0.74969
b_accuracy,0.74969
f1,0.69565


wandb: 
wandb: Plotting SVM_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.73208
auc,0.72141
b_accuracy,0.72141
f1,0.67281


wandb: 
wandb: Plotting SVM_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.6717
auc,0.66991
b_accuracy,0.66991
f1,0.6449
